In [1]:
import numpy as np
import pandas as pd

In [2]:
job_df = pd.read_csv("D:/Candidate-Matching-main/data/cleaned_recommendation.csv")

In [3]:
job_df.head(2)

,uniq_id,jobtitle,company,jobdescription,advertiserurl
0,90d755eb5b34547c31978ce7a6eb519f,full stack developer,Saicon Consultants Inc.,proficient in restful web servicesproficient i...,https://www.dice.com/jobs/detail/Full-Stack-De...
1,4b072aab5c96491c2d7f6731772b8825,full stack java engineer,RedRiver Systems L.L.C.,opportunity for an open / full stack java engi...,https://www.dice.com/jobs/detail/Full-Stack-Ja...


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
job_df['clean_text'] = job_df['jobtitle']+" "+job_df['jobdescription']

In [6]:
tfidf = TfidfVectorizer(stop_words='english')
matrix = tfidf.fit_transform(job_df['clean_text'])
similarity = cosine_similarity(matrix)

In [7]:
similarity

array([[1.        , 0.09169937, 0.09633015, ..., 0.0441837 , 0.03547016,
        0.13621698],
       [0.09169937, 1.        , 0.11257277, ..., 0.06774655, 0.1041141 ,
        0.11933016],
       [0.09633015, 0.11257277, 1.        , ..., 0.08128874, 0.05388107,
        0.13560553],
       ...,
       [0.0441837 , 0.06774655, 0.08128874, ..., 1.        , 0.0644002 ,
        0.11406638],
       [0.03547016, 0.1041141 , 0.05388107, ..., 0.0644002 , 1.        ,
        0.04542798],
       [0.13621698, 0.11933016, 0.13560553, ..., 0.11406638, 0.04542798,
        1.        ]])

In [8]:
sorted(list(enumerate(similarity[0])), key=lambda x: x[1], reverse=True)[1:20]

[(356, 0.21647200879416972),
 (124, 0.18728896818333113),
 (220, 0.18727122072763602),
 (263, 0.18463241474535402),
 (272, 0.1769192904878346),
 (469, 0.17616386290430153),
 (131, 0.1749052295664906),
 (562, 0.16992223887523597),
 (312, 0.1669008704505569),
 (216, 0.16482439669919602),
 (607, 0.16408240409002428),
 (724, 0.1638867117151132),
 (348, 0.16227296138113606),
 (20, 0.15754384784544598),
 (454, 0.15745565387091612),
 (352, 0.1566962699112247),
 (75, 0.1558019965307381),
 (111, 0.1556224163684578),
 (633, 0.155534918787598)]

# Recommendation System

In [9]:
def recommend(title):
    title = title.lower()
    if title not in job_df['jobtitle'].str.lower().values:
        return f"The job title '{title}' does not exist in the dataset."

    # Get the index of the job title
    indx = job_df[job_df['jobtitle'].str.lower() == title].index[0]
    indx = job_df.index.get_loc(indx)

    # Compute the distances and get recommendations
    distances = sorted(list(enumerate(similarity[indx])), key=lambda x: x[1], reverse=True)[1:20]

    # Create a list to store job details
    jobs = []
    for i in distances:
        job_details = {
            'uniq_id': job_df.iloc[i[0]].uniq_id,
            'jobtitle': job_df.iloc[i[0]].jobtitle,
            'company': job_df.iloc[i[0]].company,
            'jobdescription': job_df.iloc[i[0]].jobdescription,
            'advertiserurl': job_df.iloc[i[0]].advertiserurl
        }
        jobs.append(job_details)
    
    return jobs




In [10]:
recommend('Full Stack Developer')

[{'uniq_id': '1a84daa8f121b212e36750b6f64d09c8',
  'jobtitle': 'full stack java developer',
  'company': 'Kforce Inc.',
  'jobdescription': 'responsibilities:a kforce client in plano, texas (tx) is looking to hire full stack java developers.technology stack:* backend: java, spring, restful services using jersey/jax-rs* frontend: react js, angular js, bootstrap (at least one of these frameworks)essential functions:* an innovative and resourceful engineer who are passionate about creating high-quality, high performance and highly available software* love to develop highly scalable web applications and services that deliver delight to our customers* ready to roll up your sleeves and tackle technical problems and deliver products that delight customers* excited by working on a startup-like team, contributing to the vision and execution of cutting-edge products that solves problems and provides important benefits for our customers* display leadership qualities for other team members, includ

In [11]:
import pickle
pickle.dump(job_df,open('df.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [12]:
import pandas as pd
print(pd.__version__)


2.2.2
